In [2]:
import numpy as np
import scipy.linalg as scipy

In [3]:
def new_rand_A(size_n, size_m):
    A = np.zeros((size_n,size_m))
    for i in range(size_n):
        for j in range(size_m):
            A[i][j] = np.random.randint(low=0, high=9)
    
    return A

A_10 = new_rand_A(10,8)
A_100 = new_rand_A(100,100)

def secretly_rank_k(size_n, size_k):
    a = np.zeros((size_n,size_k))
    b = np.zeros((size_k, size_n))
    
    for i in range(size_n):
        for j in range(size_k):
            a[i][j] = np.random.randint(low=-9, high=9)
            b[j][i] = np.random.randint(low=-9, high=9)
    
    part = int(size_n/2)
    choice = np.random.choice(range(1,part))
    return np.matmul(a, b)[:,choice:]

def pivot_A(A, p):
    new_A = np.zeros((len(A),len(A[0])))
    for col in range(len(p)):
        new_A[:,col] = A[:,p[col]]
    return new_A

def Swap(arr, start_index, last_index):
    arr[:, [start_index, last_index]] = arr[:, [last_index, start_index]]

S_100 = secretly_rank_k(100, 3)
Q,R,p = scipy.qr(A_10, pivoting=True)
u,s,v = np.linalg.svd(S_100)


In [4]:
def apx_rank_k(size_n, rank, TOL_ord):
    a = np.zeros((size_n,size_n))
    b = np.zeros((size_n, size_n))
    for i in range(size_n):
        for j in range(size_n):
            a[i][j] = np.random.randint(low=-9, high=9)
            b[j][i] = np.random.randint(low=-9, high=9)
            
    U,R = scipy.qr(a)
    V,R = scipy.qr(b)
    TOL_ord = TOL_ord- 1
    rang = np.linspace(rank-TOL_ord,-size_n + rank - TOL_ord, size_n)
    singular_vals = []
    for i in rang:
        singular_vals.append(10**i)
    s = np.diag(singular_vals)
    matrix = np.matmul(U,s,V)
    return matrix

In [7]:
A = new_rand_A(100,95)
S = secretly_rank_k(100, 10)
A_2 = np.zeros((3,3))
ev = 2
for i in range(2):
    for j in range(3):
        A_2[i][j] = ev
        ev = ev*2
A_2[2,0] = 1
A_2[2,1] = 2
A_2[2,2] = 3

In [8]:
def RRQR_1(A, TOL=None):
    Q,R,p = scipy.qr(A, pivoting=True)          #get qr decomposition with pivoting from scipy
    pivoted_A = pivot_A(A,p)                    #pivot columns of A based on QR decomp
    k = 1                                       #starting rank = 1
    r_1 = R[0,:]                                #starting slices of the QR decomp, not important just for reference
    r_11 = R[0,0]
    r_22 = R[1:,1:]
    q_1 = Q[:,0]
    q_2 = Q[:,1:]
    
    if TOL == None:                             #default tolerance if none is given
        TOL = 10e-5
    
    err = np.linalg.norm(r_22, ord=2)           #error is equal to norm of the part of the approximated matrix that is not used
    while err > TOL and k<min(len(A), len(A[0])):
        #print("current rank, error",k, err)                              #while error is greater than tolerance, increase
        k += 1                                  #increase rank
        
        r_1 = R[:k,:]                           #update slices
        r_11 = R[:k,:k]
        r_22 = R[k:,k:]
        q_1 = Q[:,:k]
        q_2 = Q[:,k:]
        
        if len(r_11[0]) == len(R[0]):           #if approximate array is max length of R, can't update error
            None
        else:
            err = np.linalg.norm(r_22, ord=2)
        
    #print("final error: ",np.linalg.norm(r_22, ord=2))
    
    return k

k = RRQR_1(A_2, None)
print("estimated rank",k)
print("actual rank", np.linalg.matrix_rank(A_2))

k = RRQR_1(S, None)
print("estimated rank",k)
print("actual rank", np.linalg.matrix_rank(S))
'''Q,R,k = RRQR(S_100, None)
print(k)'''

estimated rank 2
actual rank 2
estimated rank 10
actual rank 10


'Q,R,k = RRQR(S_100, None)\nprint(k)'

In [10]:
'''TEST CELL FOR ABOVE'''

'''for i in range(10000):
    counter = 0
    random_m = np.random.choice(range(50,100))
    random_k = np.random.choice(range(1,50))
    S = secretly_rank_k(random_m, random_k)
    rrqr = RRQR_1(S, TOL = 10e-10)
    svd = np.linalg.matrix_rank(S)
    if rrqr != svd:
        print("wrong!")
        
print("done")'''

apx = apx_rank_k(100, 20, 5)
rrqr = RRQR_1(apx, TOL = 7e-5)
print(np.linalg.matrix_rank(apx))
print(rrqr)

14
20


In [213]:
'''THIRD TRY HAHA'''

def RRQR_2(A, TOL=None):
    m = len(A)                  #number of rows
    n = len(A[0])               #number of cols
    colnrm = np.zeros(n)
    for j in range(n):          #insert values into column norm vector
        colnrm[j] = np.linalg.norm(A[:,j], ord=2)
    
    max_col = max(colnrm)
    max_col_index = int(np.where(colnrm == max_col)[0])
    
    if TOL == None:                             #arbitrary tolerance if none is given
        TOL = 10e-5
    
    j = 0
    while j < n and max_col > TOL:
        j += 1
        Swap(A,j,max_col_index)
        
        max_col = max(colnrm[j:])
        max_col_index = int(np.where(colnrm == max_col)[0])
    
RRQR_2(A, None)

IndexError: index 95 is out of bounds for axis 1 with size 95

In [ ]:
'''interpolative decomposition whee'''

def interdecomp(A, k) '''this k here MUST be the same k that we previously found in our algorithms :)'''
{
`   '''we want to start with QR '''
    Q, R = scipy.linalg.qr(A);
    '''separating our R matrix into R11 and R12; we are disregarding the R22 this time around because its not actually part of the factorization'''
    R11 = R[:k,:k];
    R12 = R[:k,(k+1):]; '''we still have k rows, but we want to have k+1 to the end columns'''
    '''now our T is equal to R11^-1 times R12 and our I is just an identity matrix of size k and A1 is the first k columns of our matrix A'''
    '''interpolative decomp = A1 * I_k + T'''
    A1 = A[:,:k] '''all the rows but k columns'''
    R11inverse  = np.linalg.inv(R11);
    T = np.dot(R11inverse, R12);
    Ik = numpy.identity(k);
    return A1, Ik, T;      
}